# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [1]:
import sparker
import random

In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-04-04 21:33:45,698 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-04 21:33:45,699 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-04 21:33:46,058 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [4]:
# Profiles contained in the first dataset
profiles = sparker.JSONWrapper.load_profiles('/Final_Project/sample/Df2.json', real_id_field = "id")

Let's visualize a profile to check if they are correctly loaded

In [5]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'title_right', 'value': 'null , 495906 b21 hp x5560 2 80ghz ml350 g6'}, {'key': 'description_right', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s part 495906 b21'}, {'key': 'brand_right', 'value': 'hp enterprise'}, {'key': 'price_right', 'value': 'usd 213 85'}, {'key': 'specTableContent_right', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'}, {'key': 'keyValuePairs_right', 'value': "{'generation': 'g6', 'process technology': '45nm', 'processor type': 'intel xeon', 'processor core': 'quad core', 'product type': 'processor upgrade', 'bus spee

In [6]:
profiles.take(5)
profiles.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{title_right, nu...|          0|         0|        0|
|[{title_right, hp...|          1|         1|        0|
|[{title_right, as...|          2|         2|        0|
|[{title_right, as...|          3|         3|        0|
|[{title_right, as...|          4|         4|        0|
|[{title_right, as...|          5|         5|        0|
|[{title_right, as...|          6|         6|        0|
|[{title_right, as...|          8|         7|        0|
|[{title_right, as...|          9|         8|        0|
|[{title_right, as...|         10|         9|        0|
|[{title_right, as...|         11|        10|        0|
|[{title_right, as...|         14|        11|        0|
|[{title_right, as...|         15|        12|        0|
|[{title_right, as...|         16|        13|        0|
|[{title_right, as...|         17|        14|   

Extract the max id (it will be used later)

In [7]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

In [8]:
max_profile_id

38601

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [9]:
# # Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# # respectively the id of profiles of the first dataset and the id of profiles of the second dataset
# gt = sparker.CSVWrapper.load_groundtruth('../datasets/dirty/cora/groundtruth.csv', 'id1', 'id2')

In [10]:
# # Converts the groundtruth by replacing original IDs with those given by Spark
# new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [11]:
# # We can explore some pairs
# random.sample(new_gt, 2)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [12]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 63371


Let's continue by using the standard token blocking

In [13]:
blocks = sparker.Blocking.create_blocks(profiles)
print("Number of blocks",blocks.count())

Number of blocks 36403


## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [14]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [15]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [16]:
# recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

# print("Recall", recall)
# print("Precision", precision)
# print("Number of comparisons", cmp_n)

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [17]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [18]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [19]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 94706214


### Reciprocal Weighted Node Pruning

In [20]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 58309498


### Weighted Edge Pruning

In [21]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 63353287


### Cardinality Node Pruning

In [23]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.OR
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Reciprocal Cardinality Node Pruning

In [24]:
# results = sparker.CNP.cnp(
#                           blocks_after_filtering,
#                           profiles.count(),
#                           profile_blocks_filtered,
#                           block_index,
#                           max_profile_id,
#                           weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
#                           profile_blocks_size_index=profile_blocks_size_index,
#                           comparison_type=sparker.ComparisonTypes.AND
#                          )
# num_edges = results.map(lambda x: x[0]).sum()
# num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
# print("Number of comparisons",num_edges)

### Cardinality Edge Pruning

In [26]:
results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 1261379


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [29]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 11221, 47),
 (0, 11231, 50),
 (0, 11234, 47),
 (0, 19482, 50),
 (0, 19485, 47),
 (0, 20501, 47),
 (0, 20505, 47),
 (0, 20512, 47),
 (0, 20516, 50),
 (0, 14913, 47)]

In [30]:
edges_df = pd.DataFrame(edges.collect())

In [31]:
edges_df

0      1   2
0            0  20501  47
1            0  19482  50
2            0  19485  47
3            0  20505  47
4            0  20512  47
...        ...    ...  ..
1261374  25775  30076  57
1261375  25775  30084  57
1261376  25775  26326  57
1261377  25775  30444  57
1261378  25775  28592  57

[1261379 rows x 3 columns]

In [28]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         13|            0|[{0, 11221, 47}, ...|
|          1|            0|[{20512, 20516, 47}]|
|          0|            0|                  []|
|         39|            0|[{16384, 16389, 4...|
|          0|            0|                  []|
|          0|            0|                  []|
|         44|            0|[{16480, 16483, 5...|
|         22|            0|[{8304, 16439, 47...|
|          0|            0|                  []|
|          1|            0|[{16544, 20929, 1...|
|          1|            0|[{16592, 20041, 73}]|
|         34|            0|[{8432, 8441, 46}...|
|         15|            0|[{8448, 16672, 49...|
|          0|            0|                  []|
|          3|            0|[{272, 273, 52}, ...|
|          0|            0|                  []|
|         14|            0|[{16672, 22272, 4...|
|         29|       

In [ ]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [ ]:
max_by_group.take(10)

In [ ]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

In [ ]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

In [ ]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

In [ ]:

new_df = n[(n['weight']>30)]
new_df

In [ ]:
new_df = new_df.sort_values(by=['P_id'])

In [ ]:
new_df.head(20)

In [ ]:
col = ["P_id","N_id","weight"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

In [ ]:
nid_df = pd.DataFrame(new_df.iloc[:,1])
nid_df

In [ ]:
ss = SparkSession(sc)


In [ ]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .json('hdfs://localhost:9000/Final_Project/sample/Df2.json')
df1.show(5)

In [ ]:
#rdd_df = df1.rdd.zipWithIndex()

In [ ]:
# rdd_df.take(1)

In [ ]:
#df1 = rdd_df.toDF(sampleRatio=0.01)

In [ ]:
# df1.show()

In [ ]:
# df1 = df1.withColumn('brand_left', df1['_1'].getItem("brand_left"))
# df1 = df1.withColumn('description_left', df1['_1'].getItem("description_left"))
# df1 = df1.withColumn('id_left', df1['_1'].getItem("id_left"))
# df1 = df1.withColumn('identifiers_left', df1['_1'].getItem("identifiers_left"))
# df1 = df1.withColumn('keyValuePairs_left', df1['_1'].getItem("keyValuePairs_left"))
# df1 = df1.withColumn('specTableContent_left', df1['_1'].getItem("specTableContent_left"))
# df1 = df1.withColumn('title_left', df1['_1'].getItem("title_left"))
# df1 = df1.withColumn('id', df1['_1'].getItem("id"))


# df1 = df1.withColumnRenamed("_2" , "new_id_1")

In [ ]:
# df1 = df1.drop("_1")
# df1.show()

In [ ]:
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [ ]:
pd_common_df = common_df.toPandas()
pd_common_df

In [ ]:
pd_common_df = pd_common_df.sort_values(by=['P_id'])
pd_common_df

In [ ]:
pd_common_df = pd_common_df[~pd_common_df.id.isin(nid_df.N_id)]
pd_common_df

In [ ]:
pd_common_df = pd_common_df.sort_values(by=['id'])
pd_common_df

In [ ]:
pd_common_df.reset_index(inplace = True)
pd_common_df = pd_common_df.drop(["index"], axis=1)

In [ ]:
pd_common_df = pd_common_df.drop(["identifiers_right","keyValuePairs_right","P_id","N_id","weight"], axis =1)
pd_common_df

In [ ]:
pd_common_df["new_id"] = [x for x in range(0,len(pd_common_df))]
pd_common_df = pd_common_df.drop(["id"], axis=1)

In [ ]:
sp_df2 = spark.createDataFrame(pd_common_df)
sp_df2

In [ ]:
pd_common_df.to_json('/home/aditi/BigData/AAI_Project/dataset/df2_F2.json')

In [ ]:
baseDir = 'hdfs://localhost:9000/Final_Project/sample/DF_op/Df2_F2.json'
sp_df2.write.option("header", True).mode('overwrite').json(baseDir)